
# Практическая работа 4-2: Линейная регрессия 2

Name: Сачкова Галина

Email: SachkovaGG@mgpu.ru



## ##  Анализ набора кредитных данных

Вспомните набор данных «Кредит», рассмотренный на занятии и доступный [здесь](http://www-bcf.usc.edu/~gareth/ISL/data.html).
Этот набор данных состоит из информации о кредитных картах 400 человек.

Сначала импортируйте данные и преобразуйте доход в тысячи.

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name = fn, length=len(uploaded[fn])))

Saving Credit.csv to Credit.csv
User uploaded file "Credit.csv" with length 26184 bytes


In [3]:
# imports and setup

import scipy as sc
import numpy as np

import pandas as pd
import statsmodels.formula.api as sm     #Last lecture: used statsmodels.formula.api.ols() for OLS
from sklearn import linear_model         #Last lecture: used sklearn.linear_model.LinearRegression() for OLS

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

# Import data from Credit.csv file
credit = pd.read_csv('Credit.csv',index_col=0) #load data
credit["Income"] = credit["Income"].map(lambda x: 1000*x)
credit

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
1,14891.0,3606,283,2,34,11,Male,No,Yes,Caucasian,333
2,106025.0,6645,483,3,82,15,Female,Yes,Yes,Asian,903
3,104593.0,7075,514,4,71,11,Male,No,No,Asian,580
4,148924.0,9504,681,3,36,11,Female,No,No,Asian,964
5,55882.0,4897,357,2,68,16,Male,No,Yes,Caucasian,331
...,...,...,...,...,...,...,...,...,...,...,...
396,12096.0,4100,307,3,32,13,Male,No,Yes,Caucasian,560
397,13364.0,3838,296,5,65,17,Male,No,No,African American,480
398,57872.0,4171,321,5,67,12,Female,No,Yes,Caucasian,138
399,37728.0,2525,192,1,44,13,Male,No,Yes,Caucasian,0


In [4]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 1 to 400
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Income     400 non-null    float64
 1   Limit      400 non-null    int64  
 2   Rating     400 non-null    int64  
 3   Cards      400 non-null    int64  
 4   Age        400 non-null    int64  
 5   Education  400 non-null    int64  
 6   Gender     400 non-null    object 
 7   Student    400 non-null    object 
 8   Married    400 non-null    object 
 9   Ethnicity  400 non-null    object 
 10  Balance    400 non-null    int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 37.5+ KB


## Задание 1. Первая регрессионная модель

**Упражнение:** Лимит средств по рейтингу:


$$
\text{Limit} = \beta_0 + \beta_1 \text{Rating}.
$$

Поскольку кредитные рейтинги в основном используются банками для определения кредитных лимитов, ожидаем, что рейтинг будет в прямой зависимости для лимита, поэтому эта регрессия должна быть очень хорошей.

Используйте функцию `ols` из библиотеки Python `statsmodels`. Каково значение $R^2$? Что такое $H_0$ и $H_A$ для соответствующей проверки гипотезы и каково значение $p$?


**Нулевая гипотеза:** переменная Rating не имеет влияния на переменную Limit

**Альтернативная гипотеза:** переменная Rating влияет на переменную Limit



In [5]:
model = sm.ols(formula='Limit ~ Rating', data=credit).fit()
print(model.params)

Intercept   -542.928230
Rating        14.871607
dtype: float64


In [6]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Limit   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 6.348e+04
Date:                Mon, 08 Apr 2024   Prob (F-statistic):               0.00
Time:                        19:28:12   Log-Likelihood:                -2649.1
No. Observations:                 400   AIC:                             5302.
Df Residuals:                     398   BIC:                             5310.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -542.9282     22.850    -23.760      0.0

In [7]:
print(model.pvalues)

Intercept    2.467673e-78
Rating       0.000000e+00
dtype: float64


**Ваш ответ:**

Представленная модель регрессии имеет высокое значение R-квадрата(99,4%), что указывает на хорошее соответствие модели данным.

Значение p < 0.05, поэтому отвергаем нулевую гипотезу о том,  что переменная Rating не имеет влияния на переменную Limit


Кредитный рейтинг (Rating) имеет статистически значимое влияние на уровень кредитного лимита (Limit)

## Задание 2: Прогнозирование лимита без рейтинга

Поскольку `Рейтинг` и `Лимит` — это практически одна и та же переменная, исключим `Рейтинг` и просто попробуем спрогнозировать `Лимит` на основе вещественнозначных переменных (некатегориальных переменных): Доход, Карты, Возраст, Образование, Баланс.

**Упражнение.** Разработайте модель множественной линейной регрессии для прогнозирования рейтинга. Интерпретируйте результаты.

А пока просто сосредоточьтесь на переменных с численными значениями (доход, карты, возраст, образование, баланс).

В первом приближении не использовать категориальные переменные (Пол, Студент, Семейное положение, Этническая принадлежность).



In [8]:
import statsmodels.api as sm
X = credit[['Income', 'Cards', 'Age', 'Education', 'Balance']]
y = credit['Limit']
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  Limit   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.940
Method:                 Least Squares   F-statistic:                     1242.
Date:                Mon, 08 Apr 2024   Prob (F-statistic):          1.32e-238
Time:                        19:28:55   Log-Likelihood:                -3101.0
No. Observations:                 400   AIC:                             6214.
Df Residuals:                     394   BIC:                             6238.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1752.3883    170.621     10.271      0.0

Какие независимые переменные являются хорошими/плохими предикторами? Какая модель в целом является лучшей?

**Ваш ответ:**

Независимые переменные Income и Balance являются хорошими предикторами, т.к. у них coef > 0 и очень малое значение p, что является положительными факторами, так же стоить отметить Cards с низким значением p, но у этого признака отрицательный coef, это означает что с ростом этого признака Limit уменьшается. Age и Education являются плохими предикторами, т.к. имеют высокое значение p, что указывает на незначительное влияние на целевую переменную.

Модель выглядит очень хорошей, о чем свидетельствует высокое значение R-squared.


## Задание 3. Включение категориальных переменных в модель регрессии

Рассмотреть двоичные категориальные переменные, которые были ранее сопоставлены с целочисленными значениями 0, 1 в в `лекции 4.2`.

In [9]:
credit["Gender_num"] = credit["Gender"].map({' Male':0, 'Female':1})
credit["Student_num"] = credit["Student"].map({'Yes':1, 'No':0})
credit["Married_num"] = credit["Married"].map({'Yes':1, 'No':0})

Можете ли улучшить модель, которая разработана в задании `2`, включив в нее одну или несколько из этих переменных?


In [10]:
import statsmodels.formula.api as sm

credit_model = sm.ols(formula="Income ~ Gender_num", data=credit).fit()
credit_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Income   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                   0.04589
Date:                Mon, 08 Apr 2024   Prob (F-statistic):              0.830
Time:                        19:31:31   Log-Likelihood:                -4755.1
No. Observations:                 400   AIC:                             9514.
Df Residuals:                     398   BIC:                             9522.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4.561e+04   2539.977     17.957      0.000    4.06e+04    5.06e+04
Gender_num  -756.3885   3530.812     -0.214      0.830   -7697.761    6184.984
==============================================================================
Omnibus:                      137.942   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              341.093
Skew:                           1.735   Prob(JB):                     8.56e-75
Kurtosis:                       5.902   Cond. No.                         2.66
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Ваш ответ:**

Поскольку значение  p  для коэффициента Gender_num очень велико, вывод о том, что между полами существует разница в доходах, не подтверждается.